In [1]:
# QUESTION NUMBER 1
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from scipy.stats import norm
import numpy as np
import pandas as pd
import patsy
import os
import warnings
warnings.simplefilter('ignore')

In [5]:
data = pd.read_csv(r'C:\Users\Maricielo\Downloads\wage2015_subsample_inference.csv')
data = data.drop('rownames',axis=1)
list(data.columns)

['wage',
 'lwage',
 'sex',
 'shs',
 'hsg',
 'scl',
 'clg',
 'ad',
 'mw',
 'so',
 'we',
 'ne',
 'exp1',
 'exp2',
 'exp3',
 'exp4',
 'occ',
 'occ2',
 'ind',
 'ind2']

In [7]:
import pandas as pd 
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Lasso
from statsmodels.api import add_constant

# Transform categorical variables into dummies (assuming 'sex' and 'location' are categorical)
# Converting 'sex' into a binary column (0 for 'female', 1 for 'male')
data['sex'] = data['sex'].apply(lambda x: 1 if x == 'male' else 0)

# If there are other categorical variables, they can be transformed using OneHotEncoder or pandas.get_dummies
# For example, transforming a 'location' column with multiple categories:
# data = pd.get_dummies(data, columns=['location'], drop_first=True)

# Generate binary interactions
# If you need to calculate specific interactions between variables, you can do so manually or programmatically.
# Here is an example to create an interaction column between 'treat' and 'sex' (if 'treat' exists):
if 'treat' in data.columns:
    data['treat_sex_interaction'] = data['treat'] * data['sex']

# If you need a dataframe with all the original variables and the interactions:
# You can add more interactions similarly.

# Optional: Add a constant (intercept) if needed
data_with_const = add_constant(data)

print(data_with_const.head())


   const       wage     lwage  sex  shs  hsg  scl  clg   ad   mw  ...   we  \
0    1.0   9.615385  2.263364    0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0   
1    1.0  48.076923  3.872802    0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0   
2    1.0  11.057692  2.403126    0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0   
3    1.0  13.942308  2.634928    0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0   
4    1.0  28.846154  3.361977    0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0   

    ne  exp1  exp2    exp3     exp4     occ  occ2     ind  ind2  
0  1.0   7.0  0.49   0.343   0.2401  3600.0    11  8370.0    18  
1  1.0  31.0  9.61  29.791  92.3521  3050.0    10  5070.0     9  
2  1.0  18.0  3.24   5.832  10.4976  6260.0    19   770.0     4  
3  1.0  25.0  6.25  15.625  39.0625   420.0     1  6990.0    12  
4  1.0  22.0  4.84  10.648  23.4256  2015.0     6  9470.0    22  

[5 rows x 21 columns]


In [ ]:
# Identified categorical variables 
categorical_vars = ['sex', 'mw', 'so', 'we', 'ne']  # Gender and locations

# Convert categorical variables into dummies
data = pd.get_dummies(data, columns=categorical_vars, drop_first=True)

# Generate all pairwise interaction terms
interaction_terms = []
for col1 in data.columns:
    for col2 in data.columns:
        # Ensure no duplicate interaction terms are created
        if col1 != col2 and f"{col2}*{col1}" not in interaction_terms:
            data[f"{col1}*{col2}"] = data[col1] * data[col2]
            interaction_terms.append(f"{col1}*{col2}")


In [ ]:
# Select the target variable and predictors
y = data['wage']  # Salary as the dependent variable
X = data.drop(columns=['wage'])  # Remaining variables as predictors

# Scale the predictor variables for Lasso
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
# Set up cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Fit Lasso model with optimal lambda selection
lasso = LassoCV(cv=cv, random_state=42)
lasso.fit(X_scaled, y)

# Extract coefficients
lasso_coefficients = pd.Series(lasso.coef_, index=X.columns)

# Display selected coefficients
print("Selected coefficients by Lasso:")
print(lasso_coefficients[lasso_coefficients != 0])


In [ ]:
# Refit the model with selected variables
selected_features = lasso_coefficients[lasso_coefficients != 0].index
X_selected = add_constant(X[selected_features])  # Add constant for regression

ols_model = OLS(y, X_selected).fit()

# Model summary
print(ols_model.summary())

# Interpretation: Analyze coefficients
impact_clg = lasso_coefficients.filter(like='CLG')  # Filter related to CLG
print("\nImpact of being a college graduate in different groups:")
print(impact_clg)


In [ ]:
pip install networkx matplotlib

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def draw_dag(edges, title):
    """
    Draws a Directed Acyclic Graph (DAG) based on the provided edges.
    
    Parameters:
    - edges: List of tuples representing directed edges (source, target)
    - title: Title of the graph
    """
    G = nx.DiGraph()
    G.add_edges_from(edges)

    plt.figure(figsize=(10, 6))
    pos = nx.spring_layout(G, seed=42)  # Layout for consistent placement
    nx.draw(G, pos, with_labels=True, node_size=3000, node_color="lightblue", font_size=10, font_weight="bold", arrowsize=15)
    plt.title(title)
    plt.show()

# DAG for the first scenario: Tabaquismo y función pulmonar
edges_tabaquismo = [
    ("Conducta individual de tabaquismo", "Volumen respiratorio forzado"),
    ("Edad", "Conducta individual de tabaquismo"),
    ("Edad", "Volumen respiratorio forzado"),
    ("Altura", "Volumen respiratorio forzado"),
    ("Sexo", "Conducta individual de tabaquismo"),
    ("Sexo", "Volumen respiratorio forzado")
]
draw_dag(edges_tabaquismo, "Efecto del Tabaquismo sobre la Función Pulmonar")

# DAG for the second scenario: Lactancia materna e infecciones
edges_lactancia = [
    ("Lactancia materna", "Número de infecciones del bebé"),
    ("Ingresos familiares", "Lactancia materna"),
    ("Ingresos familiares", "Número de infecciones del bebé"),
    ("Educación", "Lactancia materna"),
    ("Educación", "Número de infecciones del bebé"),
    ("Número de niños en el hogar", "Lactancia materna"),
    ("Número de niños en el hogar", "Número de infecciones del bebé"),
    ("Cuidado de niños fuera del hogar", "Número de infecciones del bebé"),
    ("Cuidado de niños fuera del hogar", "Lactancia materna"),
    ("Estado civil", "Lactancia materna"),
    ("Estado civil", "Número de infecciones del bebé")
]
draw_dag(edges_lactancia, "Efecto de la Lactancia Materna sobre las Infecciones")


In [ ]:
##Parte 2

In [ ]:
import pandas as pd 
from sklearn.preprocessing import OneHotEncoder

# Load the Hitters dataset from the ISLR package online
url = "https://raw.githubusercontent.com/sahirbhatnagar/ISLR/master/data/Hitters.csv"
data = pd.read_csv(url)

# Check the first few rows of the original dataset
print("First few rows of the original dataset:")
print(data.head())

# Drop missing observations (rows with NaN values)
data_cleaned = data.dropna()

# Transform categorical variables into dummy variables (one-hot encoding)
data_transformed = pd.get_dummies(data_cleaned, drop_first=True)

# Show a summary of the transformed dataset
print("\nTransformed data:")
print(data_transformed.head())

# General information about the transformed dataset
print("\nInformation about the transformed data:")
print(data_transformed.info())


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load and clean the Hitters dataset
url = "https://raw.githubusercontent.com/sahirbhatnagar/ISLR/master/data/Hitters.csv"
data = pd.read_csv(url)

# Drop missing observations
data_cleaned = data.dropna()

# Transform categorical variables into dummy variables (one-hot encoding)
data_transformed = pd.get_dummies(data_cleaned, drop_first=True)

# Split the data into training set (90%) and test set (10%)
train_data, test_data = train_test_split(data_transformed, test_size=0.1, random_state=42)

# Check the sizes of the datasets
print(f"Size of the training set: {len(train_data)}")
print(f"Size of the test set: {len(test_data)}")

# Optional: Preview the first few rows of each dataset
print("\nFirst few rows of the training set:")
print(train_data.head())

print("\nFirst few rows of the test set:")
print(test_data.head())

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.utils import resample
from scipy.stats import scoreatpercentile

# Load and prepare the data
url = "https://raw.githubusercontent.com/sahirbhatnagar/ISLR/master/data/Hitters.csv"
data = pd.read_csv(url)
data_cleaned = data.dropna()  # Remove rows with missing values
data_transformed = pd.get_dummies(data_cleaned, drop_first=True)  # Transform categorical variables

# Split into training (90%) and test (10%) sets
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data_transformed, test_size=0.1, random_state=42)

# Split into X (features) and y (target)
X_train = train_data.drop(columns=["Salary"])
y_train = train_data["Salary"]
X_test = test_data.drop(columns=["Salary"])
y_test = test_data["Salary"]

# Fit the linear regression model on the training set
model = LinearRegression()
model.fit(X_train, y_train)
beta_hat = model.coef_

# Prediction and calculation of out-of-sample MSE
y_pred = model.predict(X_test)
mse_out_sample = mean_squared_error(y_test, y_pred)

print(f"Out-of-sample MSE: {mse_out_sample:.2f}")

# Bootstrap to estimate distributions of coefficients
n_iterations = 10000
n_samples = X_train.shape[0]
bootstrap_betas = []

for _ in range(n_iterations):
    X_resampled, y_resampled = resample(X_train, y_train, n_samples=n_samples, random_state=None)
    model_bootstrap = LinearRegression()
    model_bootstrap.fit(X_resampled, y_resampled)
    bootstrap_betas.append(model_bootstrap.coef_)

# Convert to matrix for analysis
bootstrap_betas = np.array(bootstrap_betas)

# Calculate empirical confidence intervals
percentiles = [2.5, 97.5]
ci_lower = np.percentile(bootstrap_betas, percentiles[0], axis=0)
ci_upper = np.percentile(bootstrap_betas, percentiles[1], axis=0)

# Display results
print("\nEstimated coefficients (OLS) and confidence intervals:")
for idx, col in enumerate(X_train.columns):
    print(f"{col}: β = {beta_hat[idx]:.4f}, 95% CI = [{ci_lower[idx]:.4f}, {ci_upper[idx]:.4f}]")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

# Load and prepare the data
url = "https://raw.githubusercontent.com/sahirbhatnagar/ISLR/master/data/Hitters.csv"
data = pd.read_csv(url)
data_cleaned = data.dropna()  # Remove rows with missing values
data_transformed = pd.get_dummies(data_cleaned, drop_first=True)  # Convert categorical variables to dummies

# Split into training (90%) and test (10%) sets
train_data, test_data = train_test_split(data_transformed, test_size=0.1, random_state=42)

# Split into X (features) and y (target)
X_train = train_data.drop(columns=["Salary"])
y_train = train_data["Salary"]
X_test = test_data.drop(columns=["Salary"])
y_test = test_data["Salary"]

# Fit a regression tree with pruning
best_alpha = None
best_score = float("inf")
alphas = np.linspace(0, 0.02, 50)  # Range of alphas for pruning

for alpha in alphas:
    tree = DecisionTreeRegressor(ccp_alpha=alpha, random_state=42)
    scores = -cross_val_score(tree, X_train, y_train, cv=5, scoring="neg_mean_squared_error")
    avg_score = scores.mean()  # Average cross-validation score
    if avg_score < best_score:
        best_score = avg_score
        best_alpha = alpha  # Store the best alpha for pruning

# Fit the pruned tree with the best alpha
tree_pruned = DecisionTreeRegressor(ccp_alpha=best_alpha, random_state=42)
tree_pruned.fit(X_train, y_train)

# Prediction and MSE calculation for the pruned tree
y_pred_tree = tree_pruned.predict(X_test)
mse_tree = mean_squared_error(y_test, y_pred_tree)

print(f"Best ccp_alpha (pruning parameter): {best_alpha}")
print(f"Out-of-sample MSE of pruned tree: {mse_tree:.2f}")

# Compare with the linear regression model
model_linear = LinearRegression()
model_linear.fit(X_train, y_train)
y_pred_linear = model_linear.predict(X_test)
mse_linear = mean_squared_error(y_test, y_pred_linear)

print(f"Out-of-sample MSE of linear regression model: {mse_linear:.2f}")

# Determine which model has the best performance
if mse_tree < mse_linear:
    print("The regression tree has better predictive accuracy.")
else:
    print("The linear regression model has better predictive accuracy.")

In [ ]:
###The model with the best performance in terms of predictive accuracy is the one with the lowest Out-of-Sample Mean Squared Error (MSE), as MSE measures how much the model deviates from the actual values, heavily penalizing large errors.